# Обучение RL для балансировки Cartpole

Этот ноутбук является частью [Учебной программы AI для начинающих](http://aka.ms/ai-beginners). Он был вдохновлен [официальным руководством PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) и [этой реализацией Cartpole на PyTorch](https://github.com/yc930401/Actor-Critic-pytorch).

В этом примере мы будем использовать RL для обучения модели, которая сможет балансировать шест на тележке, движущейся влево и вправо по горизонтальной шкале. Мы будем использовать среду [OpenAI Gym](https://www.gymlibrary.ml/) для симуляции этого процесса.

> **Note**: Вы можете запустить код этого урока локально (например, из Visual Studio Code), в этом случае симуляция откроется в новом окне. При запуске кода онлайн, возможно, потребуется внести некоторые изменения в код, как описано [здесь](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Начнем с проверки установки Gym:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Теперь давайте создадим среду CartPole и посмотрим, как с ней работать. Среда обладает следующими свойствами:

* **Action space** — это набор возможных действий, которые мы можем выполнять на каждом шаге симуляции.
* **Observation space** — это пространство наблюдений, которые мы можем получить.


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Давайте посмотрим, как работает симуляция. Следующий цикл запускает симуляцию до тех пор, пока `env.step` не вернет флаг завершения `done`. Мы будем случайным образом выбирать действия с помощью `env.action_space.sample()`, что, скорее всего, приведет к быстрому провалу эксперимента (среда CartPole завершается, если скорость CartPole, его положение или угол выходят за определенные пределы).

> Симуляция откроется в новом окне. Вы можете запустить код несколько раз и посмотреть, как он себя ведет.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Вы можете заметить, что наблюдения содержат 4 числа. Это:
- Положение тележки
- Скорость тележки
- Угол наклона шеста
- Скорость вращения шеста

`rew` — это награда, которую мы получаем на каждом шаге. В среде CartPole вы получаете 1 очко за каждый шаг симуляции, и цель состоит в том, чтобы максимизировать общий результат, то есть время, в течение которого CartPole может балансировать, не падая.

Во время обучения с подкреплением наша цель — обучить **политику** $\pi$, которая для каждого состояния $s$ будет определять, какое действие $a$ следует предпринять, то есть, по сути, $a = \pi(s)$.

Если вы хотите вероятностное решение, вы можете рассматривать политику как возвращающую набор вероятностей для каждого действия, то есть $\pi(a|s)$ будет означать вероятность того, что мы выберем действие $a$ в состоянии $s$.

## Метод градиента политики

В самом простом алгоритме обучения с подкреплением, называемом **Градиент политики**, мы будем обучать нейронную сеть предсказывать следующее действие.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Мы будем обучать сеть, проводя множество экспериментов и обновляя нашу сеть после каждого запуска. Давайте определим функцию, которая будет проводить эксперимент и возвращать результаты (так называемый **трейс**) - все состояния, действия (и их рекомендуемые вероятности) и награды:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Вы можете запустить один эпизод с необученной сетью и заметить, что общий награда (или длина эпизода) очень мала:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Один из сложных аспектов алгоритма градиента политики — использование **дисконтированных вознаграждений**. Идея заключается в том, что мы вычисляем вектор общих вознаграждений на каждом шаге игры, и в процессе этого дисконтируем ранние вознаграждения с использованием некоторого коэффициента $gamma$. Мы также нормализуем полученный вектор, потому что будем использовать его как вес для влияния на наше обучение:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Теперь приступим к обучению! Мы проведем 300 эпизодов, и в каждом эпизоде будем выполнять следующие шаги:

1. Запустим эксперимент и соберем трассировку.
2. Вычислим разницу (`gradients`) между выполненными действиями и предсказанными вероятностями. Чем меньше разница, тем больше уверенности в том, что было выбрано правильное действие.
3. Рассчитаем дисконтированные награды и умножим градиенты на эти награды — это обеспечит, что шаги с более высокими наградами окажут большее влияние на конечный результат, чем шаги с низкими наградами.
4. Ожидаемые целевые действия для нашей нейронной сети будут частично взяты из предсказанных вероятностей во время выполнения, а частично из рассчитанных градиентов. Мы будем использовать параметр `alpha`, чтобы определить, в какой степени учитывать градиенты и награды — это называется *скоростью обучения* алгоритма подкрепления.
5. Наконец, мы обучим нашу сеть на состояниях и ожидаемых действиях, а затем повторим процесс.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Теперь давайте запустим эпизод с рендерингом, чтобы увидеть результат:


In [ ]:
_ = run_episode(render=True)

Надеюсь, вы заметили, что теперь штанга может довольно хорошо балансировать!

## Модель Actor-Critic

Модель Actor-Critic представляет собой дальнейшее развитие градиентов политики, в которой мы создаем нейронную сеть для изучения как политики, так и предполагаемых вознаграждений. Сеть будет иметь два выхода (или можно рассматривать это как две отдельные сети):
* **Actor** будет рекомендовать действие, которое нужно выполнить, предоставляя распределение вероятностей состояний, как в модели с градиентами политики.
* **Critic** будет оценивать, какое вознаграждение можно ожидать от этих действий. Он возвращает общее предполагаемое вознаграждение в будущем для данного состояния.

Давайте определим такую модель:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Нам нужно будет немного изменить наши функции `discounted_rewards` и `run_episode`:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Теперь мы запустим основной цикл обучения. Мы будем использовать процесс ручного обучения сети, вычисляя соответствующие функции потерь и обновляя параметры сети:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Основные выводы

В этом демонстрационном примере мы рассмотрели два алгоритма обучения с подкреплением: простой градиент политики и более сложный актор-критик. Вы могли заметить, что эти алгоритмы работают с абстрактными понятиями состояния, действия и награды, что позволяет применять их к совершенно различным средам.

Обучение с подкреплением дает возможность находить лучшую стратегию для решения задачи, основываясь только на конечной награде. Тот факт, что нам не нужны размеченные наборы данных, позволяет многократно повторять симуляции для оптимизации наших моделей. Однако в области обучения с подкреплением все еще существует множество сложностей, с которыми вы можете познакомиться, если решите углубиться в эту увлекательную сферу искусственного интеллекта.



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
